# DeepLabCut Toolbox
https://github.com/AlexEMG/DeepLabCut

This notebook demonstrates the necessary steps to use DeepLabCut

Note: You can change the .ymal file to customrize your own setting and features.



## Create a new project

It is always good idea to keep the projects seperate. This function creates a new project with subdirectories and a basic configuration file in the user defined directory otherwise the project is created in the current working directory.

You can always add new videos to the project at any stage of the project. 

### NOTE: USE PYTHON 3.6 AND REMEMBER TO CONDA INSTALL JUPYTER TO RUN INSIDE PARTICULAR ENVIRONMENT

In [1]:
import deeplabcut

ModuleNotFoundError: No module named 'deeplabcut'

Change to your own working directory

In [2]:
import os
os.chdir('/home/donghan/DeepLabCut/data')

Rotate all the behavior video in order to conveniently annotate 

In [15]:
import numpy as np
import cv2
import glob
os.chdir("/home/donghan/DeepLabCut/data") 
#Working directory that stores video data
def rotate(image, angle, center=None, scale=1): 
    #scale = 1: original size
    rows,cols,ch = image.shape
    if center == None:
        center = (cols / 2, rows / 2)
    M = cv2.getRotationMatrix2D(center, angle, scale) 
    #Matrix: Rotate with center by angles
    dst = cv2.warpAffine(image,M,(cols,rows)) 
    #After rotation
    return dst


def videorotate(filename, output_name, display_video = False):
    # capture video
    cap = cv2.VideoCapture(filename)

    #read video frame by frame
    #extract original video frame features
    sz = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
            int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))

    fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))

    fps = int(cap.get(cv2.CAP_PROP_FPS))

    #Make a directory to store the rotated videos
    path = "./rotated"
    try:  
        os.mkdir(path)
    except OSError:  
        pass
    else:  
        print ("Successfully created the directory %s " % path)
        
    #Automatically name the rotated videos  
    file = "./rotated/" + output_name
    out = cv2.VideoWriter(file, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), fps, sz) 
    #Integrate all frames to video

    
    #Read videos and rotate by certain degrees
    while(cap.isOpened()):
        #flip for truning(fliping) frames of video
        ret,img = cap.read()
        try:
            img2 = rotate(img, -4.5) 
            #Flipped Vertically
            out.write(img2)
            if display_video == True:
                cv2.imshow('rotated video',img2) 

            k=cv2.waitKey(30) & 0xff
            #once you inter Esc capturing will stop
            if k==27:
                break
        except:
            print (filename, 'successfully rotated!!!' )
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    
    
#Generate all rotating videos    
filenames = glob.glob('*.mp4') #Return the file name with .mp4 extention 
for i in filenames:
    videorotate(i,os.path.splitext(i)[0] + " rotated.mp4")

1034 SI_B, Aug 15, 13 7 49DeepCut_resnet50_ReachingMar12shuffle1_800_labeled.mp4 successfully rotated!!!
1035 SI_B, Aug 15, 13 20 41.mp4 successfully rotated!!!
1034 SI_A, Aug 15, 13 4 27.mp4 successfully rotated!!!
1033 SI_B, Aug 15, 12 55 20.mp4 successfully rotated!!!
1035 SI_A, Aug 15, 13 17 7.mp4 successfully rotated!!!
1038 SI_A, Aug 15, 13 30 42.mp4 successfully rotated!!!
1038 SI_B, Aug 15, 13 34 6.mp4 successfully rotated!!!
1034 SI_B, Aug 15, 13 7 49.mp4 successfully rotated!!!


### Extract all mp4 files from rotated videos folder

In [3]:
#Extract mp4 files from video folder
from os import listdir
from os.path import isfile, join
cwd = os.chdir("./rotated") 
#we are using rotated videos
cwd = os.getcwd()
mp4files = [f for f in listdir(cwd) if isfile(join(cwd, f)) and os.path.splitext(f)[1] == ".mp4"]
#Get all mp4 files

### Starting a new project

Note that if there is an existing folder, depends on the situation, you can either delete it or find an alternative new name.

In [5]:
os.chdir("./rotated")

In [9]:
task='Reaching' # Enter the name of your experiment Task
experimenter='Donghan' # Enter the name of the experimenter
video="1038 SI_B, Aug 15, 13 34 6 rotated.mp4" # Enter the paths of your videos you want to grab frames from.

path_config_file=deeplabcut.create_new_project(task,experimenter,video, working_directory='/home/donghan/DeepLabCut/data/rotated',copy_videos=True) 
#change the working directory to where you want the folders created.


# The function returns the path, where your project is. 
# You could also enter this manually (e.g. if the project is already created and you want to pick up, where you stopped...)
#path_config_file = '/home/Mackenzie/Reaching/config.yaml' # Enter the path of the config file that was just created from the above step (check the folder)

Created "/home/donghan/DeepLabCut/data/rotated/Reaching-Donghan-2019-05-09/videos"
Created "/home/donghan/DeepLabCut/data/rotated/Reaching-Donghan-2019-05-09/labeled-data"
Created "/home/donghan/DeepLabCut/data/rotated/Reaching-Donghan-2019-05-09/training-datasets"
Created "/home/donghan/DeepLabCut/data/rotated/Reaching-Donghan-2019-05-09/dlc-models"
Copying the videos
/home/donghan/DeepLabCut/data/rotated/Reaching-Donghan-2019-05-09/videos/1038 SI_B, Aug 15, 13 34 6 rotated.mp4
Generated "/home/donghan/DeepLabCut/data/rotated/Reaching-Donghan-2019-05-09/config.yaml"

A new project with name Reaching-Donghan-2019-05-09 is created at /home/donghan/DeepLabCut/data/rotated and a configurable file (config.yaml) is stored there. Change the parameters in this file to adapt to your project's needs.
 Once you have changed the configuration file, use the function 'extract_frames' to select frames for labeling.
. [OPTIONAL] Use the function 'add_new_videos' to add new videos to your project (at 

## Extract frames from videos 
A key point for a successful feature detector is to select diverse frames, which are typical for the behavior you study that should be labeled.

This function selects N frames either uniformly sampled from a particular video (or folder) (algo=='uniform'). Note: this might not yield diverse frames, if the behavior is sparsely distributed (consider using kmeans), and/or select frames manually etc.

Also make sure to get select data from different (behavioral) sessions and different animals if those vary substantially (to train an invariant feature detector).

Individual images should not be too big (i.e. < 850 x 850 pixel). Although this can be taken care of later as well, it is advisable to crop the frames, to remove unnecessary parts of the frame as much as possible.

Always check the output of cropping. If you are happy with the results proceed to labeling.

In [10]:
%matplotlib inline
deeplabcut.extract_frames(path_config_file,algo='uniform',crop=False, userfeedback=False) #there are other ways to grab frames, such as by clustering 'kmeans'; please see the paper. 
#You can change the cropping to false, then delete the checkcropping part!
#userfeedback: ask if users would like to continue or stop

Config file read successfully.
Extracting frames based on uniform ...
Uniformly extracting of frames from 0.0  seconds to 165.75  seconds.

Frames were selected.
You can now label the frames using the function 'label_frames' (if you extracted enough frames for all videos).


## Label the extracted frames
### Remember to change the config.yaml, head and tail, size = 5
Only videos in the config file can be used to extract the frames. Extracted labels for each video are stored in the project directory under the subdirectory **'labeled-data'**. Each subdirectory is named after the name of the video. The toolbox has a labeling toolbox which could be used for labeling. 

In [11]:
%gui wx
deeplabcut.label_frames(path_config_file)

You can now check the labels, using 'check_labels' before proceeding. Then, you can use the function 'create_training_dataset' to create the training dataset.


**Check the labels**

Checking if the labels were created and stored correctly is beneficial for training, since labeling is one of the most critical parts for creating the training dataset. The DeepLabCut toolbox provides a function `check\_labels'  to do so. It is used as follows:

In [12]:
deeplabcut.check_labels(path_config_file) #this creates a subdirectory with the frames + your labels

Creating images with labels by Donghan.
They are stored in the following folder: /home/donghan/DeepLabCut/data/rotated/Reaching-Donghan-2019-05-09/labeled-data/1038 SI_B, Aug 15, 13 34 6 rotated_labeled.
If all the labels are ok, then use the function 'create_training_dataset' to create the training dataset!


If the labels need adjusted, you can use the refinement GUI to move them around! Check that out below.

## Create a training dataset
This function generates the training data information for DeepCut (which requires a mat file) based on the pandas dataframes that hold label information. The user can set the fraction of the training set size (from all labeled image in the hd5 file) in the config.yaml file. While creating the dataset, the user can create multiple shuffles. 

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**.

Now it is the time to start training the network!

In [13]:
deeplabcut.create_training_dataset(path_config_file)

The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


## Start training - If you want to use a CPU, continue. 
### If you want to use your GPU, you need to exit here and either work from the Docker container, your own TensorFlow installation in an Anaconda env

This function trains the network for a specific shuffle of the training dataset. 

In [20]:
deeplabcut.train_network(path_config_file, shuffle=1, saveiters=200, displayiters=10, maxiters = 600)
#Other parameters include trainingsetindex=0,gputouse=None,max_snapshots_to_keep=5,autotune=False,maxiters=None
#Detailed function explanation can be found here https://github.com/AlexEMG/DeepLabCut/blob/efa95129061b1ba1535f7361fe76e9267568a156/deeplabcut/pose_estimation_tensorflow/training.py

Config:
{'all_joints': [[0], [1], [2], [3]],
 'all_joints_names': ['Hand', 'Finger1', 'Finger2', 'Joystick'],
 'batch_size': 1,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_ReachingMay9/Reaching_Donghan95shuffle1.mat',
 'dataset_type': 'default',
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/home/donghan/anaconda3/envs/deeplabcut/lib/python3.6/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'max_input_size': 1500,
 'mean_pixel': [123.68, 116.779, 103.939],
 'metadataset': 'training-datasets/iteration-0/UnaugmentedDataSet_ReachingMay9/Documentation_data-Reaching_95shuffle1.pickle',
 'min_input_si

INFO:tensorflow:Restoring parameters from /home/donghan/anaconda3/envs/deeplabcut/lib/python3.6/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt


Restoring parameters from /home/donghan/anaconda3/envs/deeplabcut/lib/python3.6/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt
Restoring parameters from /home/donghan/anaconda3/envs/deeplabcut/lib/python3.6/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt
Restoring parameters from /home/donghan/anaconda3/envs/deeplabcut/lib/python3.6/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt


Max_iters overwritten as 600
Display_iters overwritten as 10
Save_iters overwritten as 200
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'weigh_only_present_joints': False, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': '/home/donghan/DeepLabCut/data/rotated/Reaching-Donghan-2019-05-09/dlc-models/iteration-0/ReachingMay9-trainset95shuffle1/train/snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'mirror': False, 'crop_pad': 0, 'scoremap_dir': 'test', 'dataset_type': 'default', 'use_gt_segm': False, 'batch_size': 1, 'video': False, 'video_batch': False, 'crop': True, 'cropratio': 0.4, 'minsize': 100, 'leftwidth': 400, 'rightwidth': 400, 'topheight

iteration: 10 loss: 0.2819 lr: 0.005
iteration: 10 loss: 0.2819 lr: 0.005
iteration: 10 loss: 0.2819 lr: 0.005
iteration: 20 loss: 0.0517 lr: 0.005
iteration: 20 loss: 0.0517 lr: 0.005
iteration: 20 loss: 0.0517 lr: 0.005
iteration: 30 loss: 0.0396 lr: 0.005
iteration: 30 loss: 0.0396 lr: 0.005
iteration: 30 loss: 0.0396 lr: 0.005
iteration: 40 loss: 0.0342 lr: 0.005
iteration: 40 loss: 0.0342 lr: 0.005
iteration: 40 loss: 0.0342 lr: 0.005
iteration: 50 loss: 0.0381 lr: 0.005
iteration: 50 loss: 0.0381 lr: 0.005
iteration: 50 loss: 0.0381 lr: 0.005
iteration: 60 loss: 0.0344 lr: 0.005
iteration: 60 loss: 0.0344 lr: 0.005
iteration: 60 loss: 0.0344 lr: 0.005
iteration: 70 loss: 0.0301 lr: 0.005
iteration: 70 loss: 0.0301 lr: 0.005
iteration: 70 loss: 0.0301 lr: 0.005
iteration: 80 loss: 0.0320 lr: 0.005
iteration: 80 loss: 0.0320 lr: 0.005
iteration: 80 loss: 0.0320 lr: 0.005
iteration: 90 loss: 0.0271 lr: 0.005
iteration: 90 loss: 0.0271 lr: 0.005
iteration: 90 loss: 0.0271 lr: 0.005
i

KeyboardInterrupt: 

## Start evaluating
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [21]:
deeplabcut.evaluate_network(path_config_file)

Config:
{'all_joints': [[0], [1], [2], [3]],
 'all_joints_names': ['Hand', 'Finger1', 'Finger2', 'Joystick'],
 'batch_size': 1,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_ReachingMay9/Reaching_Donghan95shuffle1.mat',
 'dataset_type': 'default',
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/home/donghan/anaconda3/envs/deeplabcut/lib/python3.6/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'max_input_size': 1500,
 'mean_pixel': [123.68, 116.779, 103.939],
 'metadataset': 'training-datasets/iteration-0/UnaugmentedDataSet_ReachingMay9/Documentation_data-Reaching_95shuffle1.pickle',
 'min_input_si

/home/donghan/DeepLabCut/data/rotated/Reaching-Donghan-2019-05-09/evaluation-results/  already exists!
/home/donghan/DeepLabCut/data/rotated/Reaching-Donghan-2019-05-09/evaluation-results/iteration-0/ReachingMay9-trainset95shuffle1  already exists!
Running  DeepCut_resnet50_ReachingMay9shuffle1_400  with # of trainingiterations: 400
INFO:tensorflow:Restoring parameters from /home/donghan/DeepLabCut/data/rotated/Reaching-Donghan-2019-05-09/dlc-models/iteration-0/ReachingMay9-trainset95shuffle1/train/snapshot-400


Restoring parameters from /home/donghan/DeepLabCut/data/rotated/Reaching-Donghan-2019-05-09/dlc-models/iteration-0/ReachingMay9-trainset95shuffle1/train/snapshot-400
Restoring parameters from /home/donghan/DeepLabCut/data/rotated/Reaching-Donghan-2019-05-09/dlc-models/iteration-0/ReachingMay9-trainset95shuffle1/train/snapshot-400
Restoring parameters from /home/donghan/DeepLabCut/data/rotated/Reaching-Donghan-2019-05-09/dlc-models/iteration-0/ReachingMay9-trainset95shuffle1/train/snapshot-400
0it [00:00, ?it/s]

Analyzing data...


20it [00:06,  3.72it/s]

Done and results stored for snapshot:  snapshot-400
Results for 400  training iterations: 95 1 train error: 13.47 pixels. Test error: 16.35  pixels.
With pcutoff of 0.1  train error: 13.47 pixels. Test error: 16.35 pixels
Thereby, the errors are given by the average distances between the labels by DLC and the scorer.
The network is evaluated and the results are stored in the subdirectory 'evaluation_results'.
If it generalizes well, choose the best model for prediction and update the config file with the appropriate index for the 'snapshotindex'.
Use the function 'analyze_video' to make predictions on new videos.
Otherwise consider retraining the network (see DeepLabCut workflow Fig 2)


## Start Analyzing videos
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [22]:
videofile_path = ['1035 SI_A, Aug 15, 13 17 7 rotated.mp4'] #Enter the list of videos to analyze.
deeplabcut.analyze_videos(path_config_file,videofile_path)

Config:
{'all_joints': [[0], [1], [2], [3]],
 'all_joints_names': ['Hand', 'Finger1', 'Finger2', 'Joystick'],
 'batch_size': 1,
 'bottomheight': 400,
 'crop': True,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_ReachingMay9/Reaching_Donghan95shuffle1.mat',
 'dataset_type': 'default',
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/home/donghan/anaconda3/envs/deeplabcut/lib/python3.6/site-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'leftwidth': 400,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'max_input_size': 1500,
 'mean_pixel': [123.68, 116.779, 103.939],
 'metadataset': 'training-datasets/iteration-0/UnaugmentedDataSet_ReachingMay9/Documentation_data-Reaching_95shuffle1.pickle',
 'min_input_si

Using snapshot-400 for model /home/donghan/DeepLabCut/data/rotated/Reaching-Donghan-2019-05-09/dlc-models/iteration-0/ReachingMay9-trainset95shuffle1
INFO:tensorflow:Restoring parameters from /home/donghan/DeepLabCut/data/rotated/Reaching-Donghan-2019-05-09/dlc-models/iteration-0/ReachingMay9-trainset95shuffle1/train/snapshot-400


Restoring parameters from /home/donghan/DeepLabCut/data/rotated/Reaching-Donghan-2019-05-09/dlc-models/iteration-0/ReachingMay9-trainset95shuffle1/train/snapshot-400
Restoring parameters from /home/donghan/DeepLabCut/data/rotated/Reaching-Donghan-2019-05-09/dlc-models/iteration-0/ReachingMay9-trainset95shuffle1/train/snapshot-400
Restoring parameters from /home/donghan/DeepLabCut/data/rotated/Reaching-Donghan-2019-05-09/dlc-models/iteration-0/ReachingMay9-trainset95shuffle1/train/snapshot-400
  0%|          | 0/3221 [00:00<?, ?it/s]

Starting to analyze %  1035 SI_A, Aug 15, 13 17 7 rotated.mp4
Loading  1035 SI_A, Aug 15, 13 17 7 rotated.mp4
Duration of video [s]:  161.05 , recorded with  20.0 fps!
Overall # of frames:  3221  found with (before cropping) frame dimensions:  480 470
Starting to extract posture


3232it [14:05,  3.71it/s]                          

Detected frames:  3221
Saving results in ....
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract any outlier frames!


## Extract outlier frames [optional step]
This is an optional step and is used only when the evaluation results are poor i.e. the labels are incorrectly predicted. In such a case, the user can use the following function to extract frames where the labels are incorrectly predicted. Make sure to provide the correct value of the "iterations" as it will be used to create the unique directory where the extracted frames will be saved.

In [28]:
deeplabcut.extract_outlier_frames(path_config_file,['1035 SI_A, Aug 15, 13 17 7 rotated.mp4'])

network parameters: DeepCut_resnet50_ReachingApr25shuffle1_800
Method  jump  found  151  putative outlier frames.
Do you want to proceed with extracting  20  of those?
If this list is very large, perhaps consider changing the paramters (start, stop, epsilon, comparisonbodyparts) or use a different method.
yes/nono
Nothing extracted, change parameters and start again...


## Refine Labels [optional step]
Following the extraction of outlier frames, the user can use the following function to move the predicted labels to the correct location. Thus augmenting the training dataset.

In [ ]:
%gui wx
deeplabcut.refine_labels(path_config_file)

In [ ]:
#Once all folders are relabeled, check them and advance. See how to check labels, above!
deeplabcut.merge_datasets(path_config_file)

## Create a new iteration of training dataset [optional step]
Following the refine labels, append these frames to the original dataset to create a new iteration of training dataset.

In [ ]:
deeplabcut.create_training_dataset(path_config_file)

## Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [23]:
deeplabcut.create_labeled_video(path_config_file,videofile_path)

  1%|          | 26/3221 [00:00<00:12, 259.64it/s]

Starting %  . ['1035 SI_A, Aug 15, 13 17 7 rotated.mp4']
Loading  1035 SI_A, Aug 15, 13 17 7 rotated.mp4 and data.
False 0 480 0 470
3221
Duration of video [s]:  161.05 , recorded with  20.0 fps!
Overall # of frames:  3221 with cropped frame dimensions:  480 470
Generating frames and creating video.


100%|██████████| 3221/3221 [00:14<00:00, 225.39it/s]


## Plot the trajectories of the analyzed videos
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
%matplotlib notebook #for making interactive plots.
deeplabcut.plot_trajectories(path_config_file,videofile_path)